### BeautifulSoup

- https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- pip3 install bs4
- bs4 사용 : html element를 selector를 사용

#### 1. 네이버 검색어 순위

- 네이버 검색어 순위를 가져와 데이터 프레임으로 만들기
- http://naver.com

In [1]:
import requests
import time
from bs4 import BeautifulSoup

In [2]:
def naver_top20():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://naver.com") # string형태로 가져옴
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select(".ah_roll .ah_l .ah_item")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ah_r").text,
            "keyword":keyword.select_one(".ah_k").text,
        }
    return df

In [3]:
naver_df = naver_top20()
naver_df

,rank,keyword
0,1,김기덕
1,2,붐붐파워
2,3,나쁜남자
3,4,추미애
4,5,개그맨 미투
5,6,뫼비우스
6,7,박수현
7,8,안희정나이
8,9,김부선
9,10,김지은 정무비서


#### 2. 다음 검색어 순위

- 다음 검색어 순위를 가져와 데이터 프레임으로 만들기
- http://daum.net

In [4]:
def daum_top10():
    df = pd.DataFrame(columns=["rank","keyword"])
    response = requests.get("http://daum.net")
    dom = BeautifulSoup(response.content, "html.parser")
    keywords = dom.select("#mArticle ol.list_hotissue.issue_row.list_mini > li")
    for keyword in keywords:
        df.loc[len(df)] = {
            "rank":keyword.select_one(".ir_wa").text.replace("위",""),
            "keyword":keyword.select_one(".link_issue").text,
        }
    return df

In [5]:
daum_df = daum_top10()
daum_df

,rank,keyword
0,1,안희정
1,2,김지은
2,3,장충기
3,4,박수현
4,5,뫼비우스
5,6,강태구
6,7,김부선
7,8,유시민
8,9,지주연
9,10,정유미


#### 3. 네이버, 다음 키워드에서 중복 키워드 출력하기

In [6]:
result = [keyword for keyword in daum_df["keyword"] if naver_df["keyword"].str.contains(keyword).any() ]
result

['안희정', '김지은', '장충기', '박수현', '뫼비우스', '김부선']

#### 4.  File download

In [7]:
def download(title, download_link):
    response = requests.get(download_link, stream=True)
    download_path = "./data/" + title
    size = 0
    with open(download_path, 'wb') as f:
        for chunk in response.iter_content(chunk_size=1024): 
            if chunk:
                size += 1024
                f.write(chunk)
    return size

In [8]:
title = "Apple.mp4"
download_link = "https://www.youtube.com/watch?v=cJZoTqtwGzY"
size = download(title, download_link)
print("download done : {} Mbyte".format(round(size/1024/1024,2)))

FileNotFoundError: [Errno 2] No such file or directory: './data/Apple.mp4'